# Ejemplo novedades en grafos con SQL Server 2019
Ejemplo de novedades en grafos con SQL Server 2019. Limitaciones actuales:

- Solo 1 grafo por bbdd
- Las tablas temporales locales o globales no pueden ser tablas de nodo o perimetrales.
- Los tipos de tabla y las variables de tabla no se pueden declarar como una tabla de nodo o perimetral.
- Las tablas de nodo y perimetrales no se pueden crear como tablas temporales con versión del sistema.
- Las tablas de nodo y perimetrales no pueden ser tablas con optimización para memoria.
- Los usuarios no pueden $from_id actualizar $to_id las columnas y de un borde mediante la instrucción UPDATE. Para actualizar los nodos a los que se conecta un borde, los usuarios tendrán que insertar - el nuevo borde que apunta a los nuevos nodos y eliminar el anterior.
- No se admiten las consultas entre bases de datos en objetos de grafo.

![graph](./media/SilentDataTruncation/graph.png)


__IMPORTANTE:__ Esta demo trabaja contra el contenedor por defecto generado en https://github.com/enriquecatala/mssql-server-samplesdb

# Ejemplo de constraints en aristas

## Creacion de NODO administrador
Supongamos que queremos introducir el rol de usuarios de administración lógico de nuestras aplicaciones

In [13]:
use master
go
set nocount ON
go
drop database if exists dbTest;
go
create database dbTest;
go
use dbTest
go
-- Creamos nodo adminitradores
DROP TABLE IF EXISTS administrator;
go
CREATE TABLE administrator (ID INTEGER PRIMARY KEY, administratorName VARCHAR(100)) AS NODE;
go
INSERT INTO administrator VALUES (1,'enriquecatala')
Go
SELECT * FROM administrator

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

...

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:01.630

$node_id_48348EC67B124C69BECDA028A7746F44,ID,administratorName
"{""type"":""node"",""schema"":""dbo"",""table"":""administrator"",""id"":0}",1,enriquecatala


El valor $node_id_HEX es un valor único generado automaticamente que identifica unequivocamente al nodo. En esa columna se almacena el valor único que identifica el elemento
```json
{"type":"node","schema":"dbo","table":"administrator","id":0}
```

## Creación de NODO Users
Aqui es donde iremos metiendo a nuestros usuarios administrados.

In [14]:
CREATE TABLE Users (ID INTEGER PRIMARY KEY, UserName VARCHAR(100)) AS NODE;
INSERT INTO Users
VALUES (1,'UsuarioAdministrado'), (2, 'UsuarioAdministrado2')
Go
SELECT * FROM Users


Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.031

$node_id_AC31D50AC641447DA0F1CBC0CF911323,ID,UserName
"{""type"":""node"",""schema"":""dbo"",""table"":""Users"",""id"":0}",1,UsuarioAdministrado
"{""type"":""node"",""schema"":""dbo"",""table"":""Users"",""id"":1}",2,UsuarioAdministrado2


Nótese el valor de id del json es:
```json
{"type":"node","schema":"dbo","table":"Users","id":0}
```

## Creación de arista autorización
Esta es una de las novedades en SQL Server 2019. Que podemos crear constraints para Edges

In [15]:
DROP TABLE IF EXISTS [Authorization]
-- Autorización siempre va en un sentido. Desde Administrator->Users
CREATE TABLE [Authorization] 
(
	CONSTRAINT EC_Authorization Connection (Administrator TO Users)
) As Edge
Go


Commands completed successfully.

Total execution time: 00:00:00.012

## Inserción en orden correcto
En el orden correcto tenemos la inserción que va desde Administrador hacia Usuario, porque es el administrador el que administra, no al reves

In [16]:
-- Intentemos insertar en orden correcto 
INSERT INTO [Authorization] ($from_id ,$to_id )
VALUES (
    (SELECT $node_id from administrator where ID=1), --> administrador
    (SELECT $node_id from Users where ID=1))         --> usuario
go
select * from [Authorization]

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.021

$edge_id_B4B650F76EC64B5C8189D4E470482BE1,$from_id_DE9DE7CB2836444B82B15667B4C98638,$to_id_6F3E08B6374248EBA020558BD991F3F1
"{""type"":""edge"",""schema"":""dbo"",""table"":""Authorization"",""id"":0}","{""type"":""node"",""schema"":""dbo"",""table"":""administrator"",""id"":0}","{""type"":""node"",""schema"":""dbo"",""table"":""Users"",""id"":0}"


## Inserción que viola la constraint
Ahora en SQL Server 2019, ya no se nos permite hacer algo que igual en nuestro modelo no deberia estar permitido, como es indicar que un usuario pueda administrar a un administrador :)

In [17]:
-- Intentemos insertar en orden correcto 
INSERT INTO [Authorization] ($from_id ,$to_id )
VALUES (
    (SELECT $node_id from Users where ID=2),           --> usuario
    (SELECT $node_id from administrator where ID=1))   --> administrador
go
-- Seguimos teniendo la relación anterior
select * from [Authorization]

: Msg 547, Level 16, State 0, Line 2
The INSERT statement conflicted with the EDGE constraint "EC_Authorization". The conflict occurred in database "dbTest", table "dbo.Authorization".

The statement has been terminated.

Commands completed successfully.

Total execution time: 00:00:00.021

$edge_id_B4B650F76EC64B5C8189D4E470482BE1,$from_id_DE9DE7CB2836444B82B15667B4C98638,$to_id_6F3E08B6374248EBA020558BD991F3F1
"{""type"":""edge"",""schema"":""dbo"",""table"":""Authorization"",""id"":0}","{""type"":""node"",""schema"":""dbo"",""table"":""administrator"",""id"":0}","{""type"":""node"",""schema"":""dbo"",""table"":""Users"",""id"":0}"


# Ejemplo de SHORTEST_PATH
La nueva función SHORTEST_PATH sirve para encontrar:
- Una ruta de acceso más corta entre dos nodos o entidades determinados
- Rutas de acceso más cortas de origen único.
- Ruta más corta de varios nodos de origen a varios nodos de destino.

Toma un patrón de longitud arbitraria como entrada y devuelve una ruta más corta que existe entre dos nodos. Esta función solo se puede usar dentro de la coincidencia. La función devuelve solo una ruta más corta entre dos nodos dados. Si existe, dos o más rutas de acceso más cortas de la misma longitud entre cualquier par de nodos de origen y de destino, la función devuelve solo una ruta de acceso que se encontró en primer lugar durante el recorrido. Tenga en cuenta que, un patrón de longitud arbitraria solo puede especificarse dentro de una función SHORTEST_PATH.



In [19]:
USE graphdemo;
go

DROP TABLE IF EXISTS likes;
DROP TABLE IF EXISTS Person;
DROP TABLE IF EXISTS Restaurant;
DROP TABLE IF EXISTS City;
DROP TABLE IF EXISTS friendOf;
DROP TABLE IF EXISTS livesIn;
DROP TABLE IF EXISTS locatedIn;

USE master;
go
DROP DATABASE IF EXISTS graphdemo;
go

-- Create a graph demo database
CREATE DATABASE graphdemo;
go

USE  graphdemo;
go

-- Create NODE tables
CREATE TABLE Person (
  ID INTEGER PRIMARY KEY,
  name VARCHAR(100)
) AS NODE;

CREATE TABLE Restaurant (
  ID INTEGER NOT NULL,
  name VARCHAR(100),
  city VARCHAR(100)
) AS NODE;

CREATE TABLE City (
  ID INTEGER PRIMARY KEY,
  name VARCHAR(100),
  stateName VARCHAR(100)
) AS NODE;

-- Create EDGE tables. 
CREATE TABLE likes (rating INTEGER) AS EDGE;
CREATE TABLE friendOf AS EDGE;
CREATE TABLE livesIn AS EDGE;
CREATE TABLE locatedIn AS EDGE;

-- Insert data into node tables. Inserting into a node table is same as inserting into a regular table
INSERT INTO Person VALUES (1,'John');
INSERT INTO Person VALUES (2,'Mary');
INSERT INTO Person VALUES (3,'Alice');
INSERT INTO Person VALUES (4,'Jacob');
INSERT INTO Person VALUES (5,'Julie');

INSERT INTO Restaurant VALUES (1,'Taco Dell','Bellevue');
INSERT INTO Restaurant VALUES (2,'Ginger and Spice','Seattle');
INSERT INTO Restaurant VALUES (3,'Noodle Land', 'Redmond');

INSERT INTO City VALUES (1,'Bellevue','wa');
INSERT INTO City VALUES (2,'Seattle','wa');
INSERT INTO City VALUES (3,'Redmond','wa');

-- Insert into edge table. While inserting into an edge table,
-- you need to provide the $node_id from $from_id and $to_id columns.
INSERT INTO likes VALUES ((SELECT $node_id FROM Person WHERE ID = 1), 
       (SELECT $node_id FROM Restaurant WHERE ID = 1),9);
INSERT INTO likes VALUES ((SELECT $node_id FROM Person WHERE ID = 2), 
      (SELECT $node_id FROM Restaurant WHERE ID = 2),9);
INSERT INTO likes VALUES ((SELECT $node_id FROM Person WHERE ID = 3), 
      (SELECT $node_id FROM Restaurant WHERE ID = 3),9);
INSERT INTO likes VALUES ((SELECT $node_id FROM Person WHERE ID = 4), 
      (SELECT $node_id FROM Restaurant WHERE ID = 3),9);
INSERT INTO likes VALUES ((SELECT $node_id FROM Person WHERE ID = 5), 
      (SELECT $node_id FROM Restaurant WHERE ID = 3),9);

INSERT INTO livesIn VALUES ((SELECT $node_id FROM Person WHERE ID = 1),
      (SELECT $node_id FROM City WHERE ID = 1));
INSERT INTO livesIn VALUES ((SELECT $node_id FROM Person WHERE ID = 2),
      (SELECT $node_id FROM City WHERE ID = 2));
INSERT INTO livesIn VALUES ((SELECT $node_id FROM Person WHERE ID = 3),
      (SELECT $node_id FROM City WHERE ID = 3));
INSERT INTO livesIn VALUES ((SELECT $node_id FROM Person WHERE ID = 4),
      (SELECT $node_id FROM City WHERE ID = 3));
INSERT INTO livesIn VALUES ((SELECT $node_id FROM Person WHERE ID = 5),
      (SELECT $node_id FROM City WHERE ID = 1));

INSERT INTO locatedIn VALUES ((SELECT $node_id FROM Restaurant WHERE ID = 1),
      (SELECT $node_id FROM City WHERE ID =1));
INSERT INTO locatedIn VALUES ((SELECT $node_id FROM Restaurant WHERE ID = 2),
      (SELECT $node_id FROM City WHERE ID =2));
INSERT INTO locatedIn VALUES ((SELECT $node_id FROM Restaurant WHERE ID = 3),
      (SELECT $node_id FROM City WHERE ID =3));

-- Insert data into the friendOf edge.
INSERT INTO friendOf VALUES ((SELECT $NODE_ID FROM Person WHERE ID = 1), (SELECT $NODE_ID FROM Person WHERE ID = 2));
INSERT INTO friendOf VALUES ((SELECT $NODE_ID FROM Person WHERE ID = 2), (SELECT $NODE_ID FROM Person WHERE ID = 3));
INSERT INTO friendOf VALUES ((SELECT $NODE_ID FROM Person WHERE ID = 3), (SELECT $NODE_ID FROM Person WHERE ID = 1));
INSERT INTO friendOf VALUES ((SELECT $NODE_ID FROM Person WHERE ID = 4), (SELECT $NODE_ID FROM Person WHERE ID = 2));
INSERT INTO friendOf VALUES ((SELECT $NODE_ID FROM Person WHERE ID = 5), (SELECT $NODE_ID FROM Person WHERE ID = 4));


-- Find Restaurants that John likes
SELECT Restaurant.name
FROM Person, likes, Restaurant
WHERE MATCH (Person-(likes)->Restaurant)
AND Person.name = 'John';

-- Find Restaurants that John's friends like
SELECT Restaurant.name 
FROM Person person1, Person person2, likes, friendOf, Restaurant
WHERE MATCH(person1-(friendOf)->person2-(likes)->Restaurant)
AND person1.name='John';

-- Find people who like a restaurant in the same city they live in
SELECT Person.name
FROM Person, likes, Restaurant, livesIn, City, locatedIn
WHERE MATCH (Person-(likes)->Restaurant-(locatedIn)->City AND Person-(livesIn)->City);

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:01.787

name
Taco Dell


name
Ginger and Spice


name
John
Mary
Alice
Jacob


## Buscar la ruta mas corta de un nodo a todos los demás


In [20]:
SELECT
   Person1.name AS PersonName, 
   STRING_AGG(Person2.name, '->') WITHIN GROUP (GRAPH PATH) AS Friends
FROM
   Person AS Person1,
   friendOf FOR PATH AS fo,
   Person FOR PATH  AS Person2
WHERE MATCH(SHORTEST_PATH(Person1(-(fo)->Person2)+))
AND Person1.name = 'Jacob'

Commands completed successfully.

Total execution time: 00:00:00.067

PersonName,Friends
Jacob,Mary
Jacob,Mary->Alice
Jacob,Mary->Alice->John


## Saltos y niveles que se recorren para encontrar dos elementos

In [24]:
SELECT PersonName, Friends, levels
FROM (  
   SELECT
       Person1.name AS PersonName, 
       STRING_AGG(Person2.name, '->') WITHIN GROUP (GRAPH PATH) AS Friends,
       LAST_VALUE(Person2.name) WITHIN GROUP (GRAPH PATH) AS LastNode,
       COUNT(Person2.name) WITHIN GROUP (GRAPH PATH) AS levels
   FROM
       Person AS Person1,
       friendOf FOR PATH AS fo,
       Person FOR PATH  AS Person2
   WHERE MATCH(SHORTEST_PATH(Person1(-(fo)->Person2)+))
   --AND Person1.name = 'Jacob'
) AS Q
WHERE Q.LastNode = 'Alice'

Commands completed successfully.

Total execution time: 00:00:00.040

PersonName,Friends,levels
Mary,Alice,1
John,Mary->Alice,2
Jacob,Mary->Alice,2
Alice,John->Mary->Alice,3
Julie,Jacob->Mary->Alice,3


In [0]:
## Buscar elementos a exactamente n posiciones

In [23]:
SELECT PersonName, Friends
FROM (
    SELECT
        Person1.name AS PersonName, 
        STRING_AGG(Person2.name, '->') WITHIN GROUP (GRAPH PATH) AS Friends,
        COUNT(Person2.name) WITHIN GROUP (GRAPH PATH) AS levels
    FROM
        Person AS Person1,
        friendOf FOR PATH AS fo,
        Person FOR PATH  AS Person2
    WHERE MATCH(SHORTEST_PATH(Person1(-(fo)->Person2){1,3}))
    --AND Person1.name = 'Jacob'
) Q
WHERE Q.levels = 2

Commands completed successfully.

Total execution time: 00:00:00.034

PersonName,Friends
John,Mary->Alice
Mary,Alice->John
Alice,John->Mary
Jacob,Mary->Alice
Julie,Jacob->Mary
